In [ ]:
import pandas as pd

PATH_TO_SALES = "./source_files/sales.csv"
PATH_TO_HOUSES = "./source_files/houses.csv"
PATH_TO_EMPLOYEES = "./source_files/employees.csv"



In [ ]:
# Task 1  - creating DataFrames
sales = pd.read_csv(PATH_TO_SALES)
houses = pd.read_csv(PATH_TO_HOUSES)
employees = pd.read_csv(PATH_TO_EMPLOYEES)

In [ ]:
# Task 2 - Extract EMP_FIRST_NAME and EMP_LAST_NAME from rows 3 to 10 (inclusive)
names = employees.loc[3:10, ['EMP_FIRST_NAME', 'EMP_LAST_NAME']]


In [ ]:
# Task 3 - Get amount of men and women among all employees
amount_by_gender = employees['EMP_GENDER'].value_counts()


In [ ]:
# Task 4 - Fill empty cells with 0 in 'SQUARE' column
houses['SQUARE'] = houses['SQUARE'].fillna(0)


In [ ]:
# Task 5 Create UNIT_PRICE column: price per 1 m2

import numpy as np

houses['UNIT_PRICE'] = np.where(
    houses['SQUARE'] == 0,
    -1,
    (houses['PRICE'] / houses['SQUARE']).round(2)
)


In [ ]:
#Task 6 sorting and saving price data to json file
import os

os.makedirs('output_files', exist_ok=True)

houses_sorted = houses.sort_values(by='PRICE', ascending=False)

houses_sorted.to_json('output_files/task_6.json', orient='records', indent=4)


In [ ]:
# Task 7 Filter women with first name 'Vera'
employees_filtered = employees[
    (employees['EMP_GENDER'] == 'Female') &
    (employees['EMP_FIRST_NAME'] == 'Vera')
]

# Number of such employees
employees_filtered.shape[0]


In [ ]:
# Task 8 Count houses with square >= 100 m2, grouped by category and subcategory
df = (
    houses[houses['SQUARE'] >= 100]
    .groupby(['HOUSE_CATEGORY', 'HOUSE_SUBCATEGORY'])
    .size()
    .reset_index(name='COUNT')
)


In [ ]:
#Task 9
import os
from fastavro import writer, parse_schema

os.makedirs('output_files', exist_ok=True)

schema = {
    "doc": "Employees named Vera and Female",
    "name": "Employees",
    "namespace": "employees.avro",
    "type": "record",
    "fields": [
        {"name": col, "type": ["null", "string"]} for col in employees_filtered.columns
    ]
}

parsed_schema = parse_schema(schema)

records = employees_filtered.astype(str).to_dict(orient='records')

with open('output_files/task_9.avro', 'wb') as out:
    writer(out, parsed_schema, records)


In [ ]:

#Task 10  Calculate average of SALESAMOUNT
avg_sale = sales['SALEAMOUNT'].mean()

# Update SALESAMOUNT column
sales['SALEAMOUNT'] = sales['SALEAMOUNT'].apply(lambda x: x + avg_sale * 0.02)


In [ ]:
#Task 11
import os

# Ensure output directory exists
os.makedirs('output_files', exist_ok=True)

# Find unsold houses (exists in houses but not in sales)
unsold_houses = houses[~houses['HOUSE_ID'].isin(sales['HOUSE_ID'])]

# Extract HOUSE_IDs
house_ids_available = unsold_houses['HOUSE_NAME'].unique().tolist()

# Save HOUSE_IDs to JSON
unsold_houses['HOUSE_ID'].to_json('output_files/task_11.json', orient='records', indent=4)

# Output the list
house_ids_available
